# サーチエンジンとしての利用手順(インデックス設計)
----
インデックスに与えるマッピング定義の内容や、 reindexの方法について説明します。

## 準備

本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [1]:
%run rc.py
%env ES_HOST={es_host}:9200

env: ES_HOST=10.24.128.93:9200


## マッピング設定


一般的なリレーショナルデータベースと同様に、Elasticsearchにも「データ型」の概念が存在します。  
例えば、date型は日付を表現し、日付の演算を適用して検索を行うことができる、というようなものです。

次の表に一部のデータ型の分類と説明を記載します。  
データ型に関する詳細はElasticsearch Referenceの[Field datatypes](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-types.html)を参照してください。



| データ型の分類 | データ型の説明 | 例 |
|----------------|----------------|----------------|
|[numeric](https://www.elastic.co/guide/en/elasticsearch/reference/current/number.html)|数値を扱うためのデータ型です。整数を示すデータ型と小数を扱うためのデータ型などがあります。<BR>整数と小数のいずれも、表現できる値の範囲や精度に応じていくつかのデータ型が用意されています。|int : -23<BR>float : 12.3|
|string|string形式のデータ型には[text](https://www.elastic.co/guide/en/elasticsearch/reference/current/text.html)と[keyword](https://www.elastic.co/guide/en/elasticsearch/reference/current/keyword.html)の2種類があり、両者の違いは、1つのデータ内で単語に分割されて検索インデックスが作成されるかという点です。<BR>例えば、文章をtext型としてelasticsearchに登録すれば、単語単位に分割されて検索インデックスが作成されるため検索時に文章内の単語を使って検索することができます。<BR>逆に、人名などのようにそれ以上分割してほしくない文字列を登録する時には、keyword型として登録することでそのままの文字列に対して検索インデックスが作成されます。<BR>※「検索インデックス」はElasticsearchの「インデックス」とは意味が異なります。<BR>「検索インデックス」は検索処理に利用される内部データを指します。| text : "このように長い文章はtext型として扱うのがよい。"<BR>keyword : "江戸幕府"
|[date](https://www.elastic.co/guide/en/elasticsearch/reference/current/date.html)|日付の表現にフォーマットされた文字列は、日付を表すデータ型としてdate型で登録することができます。|date : "2016-12-31T00:00:000Z", "2016-12-31"|
|[boolean](https://www.elastic.co/guide/en/elasticsearch/reference/current/boolean.html)|trueまたはfalseの値を扱うことができます。|boolean : true|
|[complex](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-types.html#_geo_datatypes)|JSON形式のデータは[object型](https://www.elastic.co/guide/en/elasticsearch/reference/current/object.html)として登録することができます。さらにJSONが入れ子になっているようなデータ形式のものも[nested型](https://www.elastic.co/guide/en/elasticsearch/reference/current/nested.html)として扱うことができます。|object : {"temperature":"20.0","rain":"1.5"}|
|[geo](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-types.html#_geo_datatypes)|緯度経度のペアからなる地点情報は[geo_point型](https://www.elastic.co/guide/en/elasticsearch/reference/current/geo-point.html)として扱うことができます。<BR>ある特定の地点を示すgeo_point型のほかに、領域を表現することができる[geo_shape型](https://www.elastic.co/guide/en/elasticsearch/reference/current/geo-shape.html)というデータ型もあります。|geo_point : [133.15 , 23.2]|
|[ip](https://www.elastic.co/guide/en/elasticsearch/reference/current/ip.html)|IPアドレスを表すデータはip型として扱うことができます。<BR>keyword型などで文字列として登録しておくこともできますが、<BR>ip型を明示的に指定することはソート時や範囲を指定しての検索などを行う場合に有用です。|ip : 192.168.0.1|


マッピング定義はJSON形式で記述します。  
基本的には、フィールド毎に名前とデータ型を指定するという構成になります。  
次のようなフォーマットでリクエストを送ることでマッピング定義を行うことができます。

上記の例のように"mappings"という項目の中にタイプを指定し、  
フィールドごとの設定は"properties"の中に記述していきます。  
この例では"message"というフィールドのデータ型をtext型として定義しています。

ここではフィールド名を指定してデータ型を定義していますが、  
フィールド名のパターンを指定することで複数のフィールドに対して定義を行うdynamic mappingという機能もあります。   
また、単一のインデックスに対してマッピング定義をする方法のほかに、  
ワイルドカードを使って複数のインデックスにマッピングを適用できるテンプレートという機能もあります。  

詳細はElasticsearch Referenceの[Mapping](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html)および[Dynamic Mapping](https://www.elastic.co/guide/en/elasticsearch/reference/current/dynamic-mapping.html)を参照してください。

## マッピングの設定方法と注意点

マッピング定義が存在しないフィールドは、自動で解釈され、text型とkeyword型の両方でElasticsearchに登録されます。  
例えば、数値データであるにも関わらずtext型で解釈された場合、大小比較による検索ができない等、想定と異なる動作となってしまいます。

次のような場合には、マッピング定義によりデータ型を明示的に指定する必要があります。  

- [string形式のデータに対して単語分割を行うかどうかを指定したい場合](#text-,-keyword)
- [日付のフォーマット指定を行いたい場合](#日付のフォーマット指定)
- [適切な数値型を利用したい場合](#配列)
- [geo_point型を利用する場合](#geolocation)

### text型 , keyword型

string型として解釈されたデータは、明示的に指定しない場合keyword型とtext型の両方が自動的に登録されます。  
明示的に指定することで、無駄なフィールドが登録されるのを防止し、インデックスのサイズを削減することができます。  
  
例えば「meteorological-data」インデックスの「information_daytime」フィールドに対して  
明示的にkeyword型を指定すればtext型で二重に登録されてしまうことはありません。  

具体的には下記のように設定します。

フィールドのマッピングを確認するコマンドは次の通りです。

In [2]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data*/_mapping?pretty"

{
  "meteorological-data-2015.04" : {
    "mappings" : {
      "logs" : {
        "dynamic_templates" : [
          {
            "my_strings" : {
              "match_mapping_type" : "string",
              "mapping" : {
                "type" : "keyword"
              }
            }
          }
        ],
        "properties" : {
          "@timestamp" : {
            "type" : "date"
          },
          "@version" : {
            "type" : "keyword"
          },
          "atmospheric_pressure" : {
            "type" : "float"
          },
          "date" : {
            "type" : "keyword"
          },
          "humidity_avg" : {
            "type" : "float"
          },
          "humidity_min" : {
            "type" : "float"
          },
          "information_daytime" : {
            "type" : "keyword"
          },
          "information_night" : {
            "type" : "keyword"
          },
          "location" : {
            "type" : "keyword"
          },
          "prec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25213  100 25213    0     0  3964k      0 --:--:-- --:--:-- --:--:-- 4103k


### 配列

配列内の要素はすべて同じデータ型である必要があります。  

データ型を定義する時は、単一のフィールドに対してデータ型を指定する時と同じ方法で  
配列の型を指定することができます。

たとえば  
float_array : [2.5, 4.0, -8.2]  
といったような、float型の要素を持つ配列であれば、  
下記のようにマッピング定義を登録すれば、配列の各要素に対してデータ型の指定が適用されます。

配列に関する詳細はElasticsearch Referenceの[Array datatype](https://www.elastic.co/guide/en/elasticsearch/reference/current/array.html)を参照してください。

### nested JSON


ネストされたJSON形式のデータも扱うことができます。  
例えば、下記のようなデータを扱うことを想定してマッピングの設定方法を説明します。

上の例では、userというフィールドが内部に2つのobjectを持っています。  
内部のobjectに対してデータが型指定を行いたい場合は、  
user.firstといったように、"."を使って表現することでアクセスが可能です。

マッピングは下記のようなリクエストで定義できます。

### geolocation


緯度・経度の情報からなる地点情報は、geo_point型として扱います。  
具体的なマッピング方法は下記の通りです。

geo_point型として登録されているデータに対して利用できるGeo queriesが多数用意されています。    
詳細はElasticsearch Referenceの[Geo queries](https://www.elastic.co/guide/en/elasticsearch/reference/current/geo-queries.html)を参照してください。

### 日付のフォーマット指定

マッピング定義を行う際に、日付のフォーマットを指定することができます。  
具体的な設定例を次に示します。  

日付フォーマットについての詳細は、Elasticsearch Referenceの[format](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/mapping-date-format.html)を参照してください。

## reindex APIを使った投入後データの加工方法

一度登録したデータに対して、型を変更する必要が生じた場合や、  
フィールドの値を計算した結果を登録したい場合にはreindex APIを用いることができます。  
次にreindex APIの使用例を示します。

登録してあるドキュメントは下記のものを想定します。
実行してドキュメントを登録してください。

In [3]:
%%bash
curl -XPUT "http://$ES_HOST/reindex_source/logs/1?pretty" -d @- << EOF 
{
  "max_temperature":31,
  "min_temperature":18,
  "rain":"3"
}
EOF

{
  "_index" : "reindex_source",
  "_type" : "logs",
  "_id" : "1",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "created" : true
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   268  100   208  100    60   4784   1380 --:--:-- --:--:-- --:--:--  4837


上のセルを実行することで登録したドキュメントを加工します。

- 降水量を意味する"rain"というフィールドをfloat型と誤ってkeyword型として登録してしまった
- 最高気温と最低気温の差分をフィールドとして追加したい

という状況での再加工を想定します。

ingest nodeの機能であるconvert processorを用いることで、型変換の処理を行います。  
ingest nodeに関しては[04_Store_Data](04_Store_Data.ipynb)を参照してください。

次のセルを実行することで、ingest nodeのpipelineを登録してください。

In [4]:
%%bash
curl -XPUT "http://$ES_HOST/_ingest/pipeline/convert_to_float" -d @- << EOF 
{
  "processors":[
  {
    "convert":{
      "field":"rain",
      "type" : "float"
    }
  }
  ]
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100    21  100    90   1935   8294 --:--:-- --:--:-- --:--:--  9000


下記のリクエストによって、reindex処理が実行されます。  
下の例では"reindex_source"から"reindex_dest"が作成され、"reindex_dest"の"rain"フィールドはfloatに変換されています。  
また、"gap_temperature"が追加され、最高気温と最低気温の差分が登録されています。

In [5]:
%%bash
curl -XPOST "http://$ES_HOST/_reindex?pretty" -d @- << EOF
{
  "source":{
    "index":"reindex_source"   
  },
  "dest":{
    "index":"reindex_dest",
    "version_type": "external",
    "pipeline":"convert_to_float"
  },
  "script":{
    "lang": "painless",
    "inline": "ctx._source.gap_temperature = ctx._source.max_temperature-ctx._source.min_temperature"
  }
}
EOF

{
  "took" : 87,
  "timed_out" : false,
  "total" : 1,
  "updated" : 0,
  "created" : 1,
  "deleted" : 0,
  "batches" : 1,
  "version_conflicts" : 0,
  "noops" : 0,
  "retries" : {
    "bulk" : 0,
    "search" : 0
  },
  "throttled_millis" : 0,
  "requests_per_second" : -1.0,
  "throttled_until_millis" : 0,
  "failures" : [ ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   623  100   330  100   293   3497   3105 --:--:-- --:--:-- --:--:--  3510


次のセルを実行して、reindexされたドキュメントを確認してください。  
最高気温と最低気温の差分を示す"gap_temperature"のフィールドが追加されていればreindex処理が正常に行われています。  
また、rainのデータ型をfloat型に変更したことで、値がダブルクォーテションで囲われていないことも確認してください。  

In [6]:
%%bash
curl -XGET "http://$ES_HOST/reindex_dest/_search?pretty"

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "reindex_dest",
        "_type" : "logs",
        "_id" : "1",
        "_score" : 1.0,
        "_source" : {
          "max_temperature" : 31,
          "rain" : 3.0,
          "gap_temperature" : 13,
          "min_temperature" : 18
        }
      }
    ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   475  100   475    0     0   124k      0 --:--:-- --:--:-- --:--:--  154k


reindex APIのより詳細な使い方については、Elasticsearch Referenceの[Reindex API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-reindex.html)を参照してください。

## パラメータ設定


REST APIを利用して、インデックスのパラメータ設定を変更することができます。  
ここでは、代表的なパラメータ設定と、その変更方法について説明します。  

インデックスのパラメータは下記のようなフォーマットで指定します。

### シャード・レプリカ数


インデックスの作成時にのみ設定することができるパラメータです。  
シャード数は**index.number_of_shards**、レプリカ数は**index.number_of_replicas**  
という項目で設定することができます。  

シャード数やレプリカ数を変更することにはメリットとデメリットの両面があります。  
詳細に関してはElasticsearch Referenceの[Index Modules](https://www.elastic.co/guide/en/elasticsearch/reference/current/index-modules.html#_static_index_settings)および[Index Settings](https://www.elastic.co/guide/en/elasticsearch/guide/2.x/_index_settings.html)を参照してください。

### 圧縮方式指定


データを保持する際の圧縮形式も指定することができます。  
設定を変更することで圧縮率を上げることができますが、パフォーマンスは低下する可能性があります。  
詳細に関してはElasticsearch Referenceの[Index Modules](https://www.elastic.co/guide/en/elasticsearch/reference/current/index-modules.html#_static_index_settings)を参照してください。

### 検索結果上限数

検索結果として返ってくるデータの上限数を明示的に指定して数を制限することができます。  
具体的にはリクエストの中の"size"というパラメータを指定することになります。  
実際のリクエストフォーマットは下記の通りです。

上記のようにsizeを0に指定した場合は、ヒット件数などの情報は出力されますが、  
ヒットしたドキュメントの内容は一切出力されません。  
詳細は、Elasticsearch Referenceの[From / Size](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-request-from-size.html)を参照してください。